<div align="center"><h1> Hackathon HPC SENAI CIMATEC <br>
Espacialização de Relevo </h1></div>

**Murilo Boratto**$^1$

$^1$ Supercomputing Center SENAI CIMATEC, Salvador, Bahia, Brazil

## Instalação das Bibliotecas no Colab

Este é o passo a passo de como instalar as APIs em ambiente virtual do Colab, utilizando a implementação open source.

In [1]:
!sudo apt install mpich libopenblas-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libopenblas-dev is already the newest version (0.3.20+ds-1).
The following additional packages will be installed:
  hwloc-nox libmpich-dev libmpich12 libslurm37
Suggested packages:
  mpich-doc
The following NEW packages will be installed:
  hwloc-nox libmpich-dev libmpich12 libslurm37 mpich
0 upgraded, 5 newly installed, 0 to remove and 49 not upgraded.
Need to get 14.2 MB of archives.
After this operation, 102 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libslurm37 amd64 21.08.5-2ubuntu1 [542 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 hwloc-nox amd64 2.7.0-2ubuntu1 [205 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libmpich12 amd64 4.0-3 [5,866 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 mpich amd64 4.0-3 [197 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy/universe 

In [57]:
!sudo apt-get install liblapack-dev


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
liblapack-dev is already the newest version (3.10.0-2ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


### 1 GPU (CUDA)

In [73]:
%%writefile r3d-1GPU.cu
#include <omp.h>
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <cuda_runtime.h>  // Para CUDA Runtime

#define Mxx 5026
#define Mxy 5026
#define INPUT "mde.pnz"
#define OUTPUT "output.txt"
#define ANOVA "anova.txt"

// Protótipos das funções
long int n = 0;  // Número global de pontos
double zm = 0.0;  // Média de z

void dados(double *x, double *y, double *z);
void matrizes(double *A, double *B, double *x, double *y, double *z, int N, int r);
void gauss_elimination(double *A, double *B, int N);
void anova(double *A, double *x, double *y, double *z, int N, int r);

__global__ void calculate_matrices_kernel(double *A, double *B, double *x, double *y, double *z, int N, int n, int r) {
    int l = blockIdx.x * blockDim.x + threadIdx.x;  // Índice para linhas de A
    int c = blockIdx.y * blockDim.y + threadIdx.y;  // Índice para colunas de A

    if (l < N && c < N) {
        double local_A = 0.0;
        double local_B = 0.0;

        // Cálculo das matrizes A e B
        for (int i = 0; i < n; ++i) {
            int xIdx = (int)(l / (r + 1)) + (int)(c / (r + 1));
            int yIdx = l % (r + 1) + c % (r + 1);

            local_A += pow(x[i], xIdx) * pow(y[i], yIdx);
            if (c == 0) {
                local_B += z[i] * pow(x[i], xIdx) * pow(y[i], yIdx);
            }
        }

        A[l + c * N] = local_A;
        if (c == 0) {
            B[l] = local_B;
        }
    }
}

int main(int argc, char **argv) {
    int r = atoi(argv[1]);  // Grau do polinômio
    int s = r;               // Simplify Polynomial Degree
    int N = (r + 1) * (s + 1);  // Número de coeficientes de polinômio
    int MAX = (Mxx + 1) * (Mxy + 1);

    double *A = (double*)malloc(sizeof(double) * N * N);
    double *B = (double*)malloc(sizeof(double) * N);
    double *x = (double*)malloc(sizeof(double) * MAX);
    double *y = (double*)malloc(sizeof(double) * MAX);
    double *z = (double*)malloc(sizeof(double) * MAX);

    double t1, t2;

    // Inicializa o tempo
    t1 = omp_get_wtime();

    // Etapas do processo
    dados(x, y, z);

    // Aloca memória na GPU para A, B, x, y, z
    double *d_A, *d_B, *d_x, *d_y, *d_z;
    cudaMalloc((void **)&d_A, sizeof(double) * N * N);
    cudaMalloc((void **)&d_B, sizeof(double) * N);
    cudaMalloc((void **)&d_x, sizeof(double) * n);
    cudaMalloc((void **)&d_y, sizeof(double) * n);
    cudaMalloc((void **)&d_z, sizeof(double) * n);

    // Copia dados para a memória da GPU
    cudaMemcpy(d_x, x, sizeof(double) * n, cudaMemcpyHostToDevice);
    cudaMemcpy(d_y, y, sizeof(double) * n, cudaMemcpyHostToDevice);
    cudaMemcpy(d_z, z, sizeof(double) * n, cudaMemcpyHostToDevice);

    // Definir blocos e grids para o kernel
    dim3 blockDim(16, 16);  // Blocos de 16x16 threads
    dim3 gridDim((N + blockDim.x - 1) / blockDim.x, (N + blockDim.y - 1) / blockDim.y);

    // Lançar o kernel para calcular A e B
    calculate_matrices_kernel<<<gridDim, blockDim>>>(d_A, d_B, d_x, d_y, d_z, N, n, r);

    // Sincroniza a execução da GPU
    cudaDeviceSynchronize();

    // Copiar os resultados de volta para a CPU
    cudaMemcpy(A, d_A, sizeof(double) * N * N, cudaMemcpyDeviceToHost);
    cudaMemcpy(B, d_B, sizeof(double) * N, cudaMemcpyDeviceToHost);

    // Libera a memória da GPU
    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_x);
    cudaFree(d_y);
    cudaFree(d_z);

    // Resolve o sistema linear utilizando eliminação de Gauss
    gauss_elimination(A, B, N);

    // Realiza a análise de variância (ANOVA)
    anova(B, x, y, z, N, r);

    // Finaliza o tempo
    t2 = omp_get_wtime();

    printf("%d\t%5.2f\n", r, t2 - t1);

    // Libera a memória alocada
    free(A);
    free(B);
    free(x);
    free(y);
    free(z);

    return 0;
}

void dados(double *x, double *y, double *z) {
    int col, row, count = -1;
    FILE *f;
    n = 0;  // Inicializa n aqui

    if ((f = fopen(INPUT, "r")) == NULL) {
        printf("\n Erro I/O\n");
        exit(1);
    }

    for (row = 0; row < Mxy; ++row) {
        for (col = 0; col < Mxx; ++col) {
            float h;
            int result = fscanf(f, "%f", &h);
            count++;
            if (count % 10 != 0 || h <= 0) continue;
            x[n] = row;
            y[n] = col;
            z[n] = h / 2863.0;
            n++;  // Incrementa n para o próximo ponto
        }
    }

    // Calcula a média de z para armazenar em zm
    double sum = 0.0;
    for (int i = 0; i < n; i++) {
        sum += z[i];
    }
    zm = sum / n;  // Calcula a média

    fclose(f);
}

void gauss_elimination(double *A, double *B, int N) {
    for (int k = 0; k < N; ++k) {
        // Encontrar o pivô
        double pivot = A[k + k * N];
        if (pivot == 0.0) {
            printf("Pivô nulo encontrado, o sistema é singular.\n");
            exit(1);
        }

        // Normaliza a linha
        for (int j = k + 1; j < N; ++j) {
            A[k + j * N] /= pivot;
        }
        B[k] /= pivot;

        // Elimina os elementos abaixo do pivô
        for (int i = k + 1; i < N; ++i) {
            double factor = A[i + k * N];
            for (int j = k + 1; j < N; ++j) {
                A[i + j * N] -= factor * A[k + j * N];
            }
            B[i] -= factor * B[k];
        }
    }

    // Substituição para trás
    for (int k = N - 1; k >= 0; --k) {
        for (int i = k - 1; i >= 0; --i) {
            double factor = A[i + k * N];
            A[i + k * N] = 0.0;
            B[i] -= factor * B[k];
        }
    }
}

void anova(double *A, double *x, double *y, double *z, int N, int r) {
    int i, glReg, glR, glT, c, l;
    double SQReg, SQR, SQT, QMReg, QMR, R2, F, ze;
    FILE *f;
    int s = r;

    SQR = SQReg = 0.0;
    glReg = N;
    glR = n - 2 * N;
    glT = n - N;

    for (i = 0; i < n; ++i) {  // Usando n aqui
        ze = 0.0;
        for (c = 0; c < r + 1; ++c)
            for (l = 0; l < s + 1; ++l)
                ze += A[c + l * (r + 1)] * pow(x[i], c) * pow(y[i], l);

        SQReg += (ze - zm) * (ze - zm);
        SQR += (z[i] - ze) * (z[i] - ze);
    }

    SQT = SQReg + SQR;
    QMReg = SQReg / glReg;
    QMR = SQR / glR;
    F = QMReg / QMR;
    R2 = SQReg / SQT;

    if ((f = fopen(ANOVA, "w")) == NULL) {
        printf("\n Error I/O\n");
        exit(2);
    }

    fprintf(f, " \n\n\n\n");
    fprintf(f, " ANOVA\n");
    fprintf(f, " =================================================\n");
    fprintf(f, " FV           gl      SQ         QM          F    \n");
    fprintf(f, " =================================================\n");
    fprintf(f, " Regression  %5d  %12e  %12e  %12e\n", glReg, SQReg, QMReg, F);
    fprintf(f, " Residue     %5d  %12e  %12e      \n", glR, SQR, QMR);
    fprintf(f, " -------------------------------------------------\n");
    fprintf(f, " Total      %5d  %12e            \n", glT, SQT);
    fprintf(f, " =================================================\n");
    fprintf(f, " R^2= %12e                        \n", R2);

    for (c = 0; c < r + 1; ++c)
        for (l = 0; l < s + 1; ++l) {
            fprintf(f, "+x^%d*y^%d*\t%12g\n", c, l, A[c + l * (r + 1)]);
        }

    fclose(f);
}


Overwriting r3d-1GPU.cu


In [76]:
!nvcc r3d-1GPU.cu -o r3d-1GPU -Xcompiler="-fopenmp -O3 -lm" -lcublas

In [77]:
!./r3d-1GPU 2 64

2	17.32


In [70]:
!./r3d-1GPU 8 64

8	29.69


In [68]:
!./r3d-1GPU 10 64

10	53.96


## Análise Experimental

### I) Validação com Valores Pequenos

#### Parâmetros Ótimos de Execução

1. OpenMP = **A** Threads
2. MPI = **B** nós + **C** Processos
3. MPI + OpenMP = **D** nós + **E** Processos + **F** Threads
4. CUDA = G1D B1DT1D (**G** * 32, 1024)

### Tempos de execução em segundos das aplicações

|  Grau | Sequencial Lapack| OpenMP | MPI  | Híbrido | CUDA
| ----------| ---------- | ------ | ---  | ------- | ----
| 2         |            |        |      |         |  
| 4         |            |        |      |         |  
| 6         |            |        |      |         |  
| 8         |            |        |      |         |  
| 10        |            |        |      |         |  

### Speedups

|  Grau  |  OpenMP  | MPI     | Híbrido       | CUDA
| ---------   | ------   | ------  | -------       | ----
| 2           |          |         |               |           
| 4           |          |         |               |           
| 6           |          |         |               |           
| 8           |          |         |               |           
| 10          |          |         |               |           

### II) Análise de Desempenho com um valor alto - Grau do Polinômio = `20`

#### Parâmetros Ótimos de Execução

1. OpenMP = **A** Threads
2. MPI = **B** nós + **C** Processos
3. MPI + OpenMP = **D** nós + **E** Processos + **F** Threads
4. CUDA = G1D B1DT1D (**G** * 32, 1024)

### Tempo de execução em segundos das aplicações (referência @muriloboratto)

|  Grau        | Sequencial Lapack| OpenMP  |  CUDA (1 GPU)
| -------------| ---------------- | ------- |  -----------
| 10           | 2060.14          | 62.55   |      14.72
| 20           | 27986.75         | 744.77  |      70.03


### Speedup (referência @muriloboratto)

|  Grau        |  OpenMP    |  CUDA (1GPU)
| -------------|  --------  |  -----------
| 10           |  32X       |     139X
| 20           |  37X       |     399X


## Conclusões

Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.

## Referências Biliográficas

* G. Coulouris, J. Dollimore, T. Kindberg, G.Blair. Distributed Systems: Concepts and Design, Fifth Edition, Pearson, 2011.

* S.Tanenbaum, M. Steen, Distributed Systems: Principles and Paradigms, Second Edition, Pearson, 2006.

* David A. Patterson and John L. Hennessy. Computer Organization and Design: The Hardware/Software Interface. Morgan Kaufmann, 5th Edition, 2013.

* An Introduction to Parallel Programming by Peter S. Pacheco. Morgan Kauffman.

* W. C. Barbosa, An introduction to distributed algorithms, MIT Press, 1997. N. Lynch, Distributed Algorithms, Mit Press, 1996 e Introduction to Distributed Algorithms, Gerard Tel, Cabribridge U. Press, 1994.